### 1.1. Загрузка данных о ценах, доходностях, НКД из архива cbonds_estimation_cis_2.csv

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
pd.options.display.max_rows = 999
data = pd.read_csv('cbonds_estimation_cis_2.csv', sep=';', encoding='latin-1', decimal=',')
data.columns = ['data','isin', 'id', 'price', 'ytm', 'duration', 'nkd']
data['data'] = data['data'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
from datetime import datetime, timedelta
import numpy as np
eurobonds_prices = pd.DataFrame(index=data['data'].drop_duplicates(), columns=data['isin'].drop_duplicates())
eurobonds_ytms = pd.DataFrame(index=data['data'].drop_duplicates(), columns=data['isin'].drop_duplicates())
eurobonds_durations = pd.DataFrame(index=data['data'].drop_duplicates(), columns=data['isin'].drop_duplicates())
eurobonds_nkds = pd.DataFrame(index=data['data'].drop_duplicates(), columns=data['isin'].drop_duplicates())

for index, row in data.iterrows():
    eurobonds_prices[row['isin']][row['data']] = row['price']
    eurobonds_ytms[row['isin']][row['data']] = row['ytm'] * 100
    eurobonds_durations[row['isin']][row['data']] = row['duration'] / 365
    if isinstance(row['nkd'], float) == True:
        eurobonds_nkds[row['isin']][row['data']]  = row['nkd']
    else:
        eurobonds_nkds[row['isin']][row['data']] = 0
eurobonds_prices = eurobonds_prices.sort_values('data')
eurobonds_ytms = eurobonds_ytms.sort_values('data')
eurobonds_durations = eurobonds_durations.sort_values('data')
eurobonds_nkds = eurobonds_nkds.sort_values('data')


In [ ]:
eurobonds_spreads.head(2)

In [ ]:
import requests
import matplotlib.pyplot as plt
from matplotlib import figure
import pandas as pd
import numpy as np
import time
pd.options.display.max_columns = 999
from datetime import datetime, timedelta
isins_list = [
#            'RU000A0JXU14', 
#          'RU000A0JWHA4', 
#          'RU000A0JXTS9', 
#          'RU000A0ZYYN4', 
#          'RU000A1006S9', 
#          'XS0088543193', 
#          'XS0767472458',
#          'XS0767473852',
#          'XS0971721450',
#          'XS0971721963'
    'XS0089375249',
# 'XS0088543193',
'XS0767469827',
'XS0971721377',
'XS0767472458',
'XS0971721450',
'XS0504954347',
'XS0767473852',
'XS0971721963'
             ]
eurobonds_prices = pd.DataFrame(columns=isins_list)
eurobonds_spreads = pd.DataFrame(columns=isins_list)
eurobonds_ytms = pd.DataFrame(columns=isins_list)
eurobonds_durations = pd.DataFrame(columns=isins_list)
eurobonds_nkds = pd.DataFrame(columns=isins_list)
eurobonds_info = pd.DataFrame(columns=['isin', 'maturity_date', 'coupon'])

# eurobonds_prices
for i in range(len(isins_list)):
    current_bond = pd.read_excel('quotes.xlsx', engine='openpyxl', sheet_name=isins_list[i], skiprows=6, index_col=0)  
    eurobonds_prices[isins_list[i]] = current_bond['Indicative price, %']
    eurobonds_nkds[isins_list[i]] = current_bond['ACI']
    eurobonds_durations[isins_list[i]] = current_bond['Duration'] / 360
    eurobonds_ytms[isins_list[i]] = current_bond['YTM, effective (indicative), %']
    eurobonds_spreads[isins_list[i]] = current_bond['Bid-ask spread, bp']
    dict_info = {
        'coupon':current_bond['Current coupon rate, %'].iloc[0] / 100,
        'isin':isins_list[i],
        'maturity_date': current_bond['Maturity date'].iloc[0],   
#     'coupon payments per year':2
    }
    eurobonds_info = eurobonds_info.append(dict_info, ignore_index=True)
eurobonds_spreads.index.reindex(current_bond.index)
eurobonds_spreads = eurobonds_spreads.loc[eurobonds_spreads.index.sort_values(ascending=True)]
eurobonds_spreads = eurobonds_spreads.fillna(method='bfill')
eurobonds_prices.index.reindex(current_bond.index)
eurobonds_prices = eurobonds_prices.loc[eurobonds_prices.index.sort_values(ascending=True)]
eurobonds_prices = eurobonds_prices.fillna(method='bfill')
eurobonds_nkds.index.reindex(current_bond.index)
eurobonds_nkds = eurobonds_nkds.loc[eurobonds_nkds.index.sort_values(ascending=True)]
eurobonds_nkds = eurobonds_nkds / 200000
eurobonds_nkds = eurobonds_nkds.fillna(0)
eurobonds_nkds = eurobonds_nkds.interpolate(axis=1)
eurobonds_durations.index.reindex(current_bond.index)
eurobonds_durations = eurobonds_durations.loc[eurobonds_durations.index.sort_values(ascending=True)]
eurobonds_ytms.index.reindex(current_bond.index)
eurobonds_ytms = eurobonds_ytms.loc[eurobonds_ytms.index.sort_values(ascending=True)]
eurobonds_info['coupon_payments_per_year'] = 2
eurobonds_prices

In [ ]:
eurobonds_prices = eurobonds_prices[isins_list]
eurobonds_ytms = eurobonds_ytms[isins_list]
eurobonds_durations = eurobonds_durations[isins_list]
eurobonds_nkds = eurobonds_nkds[isins_list]
eurobonds_prices = eurobonds_prices.dropna(axis=0, how='all')
eurobonds_ytms = eurobonds_ytms.loc[eurobonds_prices.index]
eurobonds_durations = eurobonds_durations.loc[eurobonds_prices.index]
eurobonds_nkds = eurobonds_nkds.loc[eurobonds_prices.index]


eurobonds_prices.head(2)

### 1.2. Загрузка данных об эмиссиях и купонных платежах


In [ ]:
isins_list = [

         'RU000A0JXU14', 
         'RU000A0JWHA4', 
         'RU000A0JXTS9', 
         'RU000A0ZYYN4', 
         'RU000A1006S9', 
         'XS0767472458',
         'XS0767473852',
         'XS0971721450',
         'XS0971721963'
             ]

In [ ]:
coupons = pd.DataFrame(columns=['isin', 'coupon_N', 'date', 'rate'])
for isin in isins_list:
    print(isin)
#     isin = 'RU000A0JXU14'
    current = pd.read_excel('Eurobonds info + coupons.xlsx', sheet_name=isin)
    skip = 0
    for index, row in current.iterrows():
        if row[0] == 'Payment schedule': 
            skip = index
            break      
    current = pd.read_excel('Eurobonds info + coupons.xlsx', engine='openpyxl', sheet_name=isin, skiprows=skip+2)   
    current = current[['Coupon date', '№', 'Coupon, %']]
    current['date'] = current['Coupon date']
    del current['Coupon date']
    current['coupon_N'] = current['№']
    del current['№']
    current['rate'] = current['Coupon, %']
    del current['Coupon, %']
    current['isin'] = isin
    coupons = coupons.append(current)
coupons = coupons[coupons['date'] > '2015-01-01']
coupons

In [ ]:
coupons = pd.DataFrame(columns=isins_list)
coupons = pd.DataFrame()
eurobonds_info = pd.DataFrame(columns=['isin', 'face_value', 'maturity_date', 
                                       'coupon', 'coupon_payments_per_year'])

for isin in isins_list:
    print(isin)
    k = 0
#     current_bond = pd.read_excel('eurobonds_info+coupons.xlsx', sheet_name=isin,
#                                 engine='openpyxl')
    current_bond = pd.read_excel('Eurobonds info + coupons.xlsx', sheet_name=isin,
                                engine='openpyxl')
    dict_bond = {
        'isin':isin,
        'face_value':current_bond.iloc[9].iloc[1],
        'maturity_date':current_bond.iloc[13].iloc[4],
        'coupon':0,
        'coupon_payments_per_year':2
    }
    
    while current_bond['Unnamed: 0'].iloc[k] != 'Payment schedule':
        k += 1
#     print(k)
#     current_coupons = pd.read_excel(os.getcwd()+'//eurobonds_info+coupons.xlsx', sheet_name=isin,
#                                 engine='openpyxl', skiprows=k+2, index_col=0)
    current_coupons = pd.read_excel(os.getcwd()+'//Eurobonds info + coupons.xlsx', sheet_name=isin,
                                engine='openpyxl', skiprows=k+2, index_col=0)
    current_coupons['Coupon size'] =  current_coupons['Coupon, %'] * dict_bond['face_value'] / 200
    current_coupons['isin'] = isin
    display(current_coupons.head(2))
    coupons = coupons.append(current_coupons)
    
    dict_bond['coupon'] = current_coupons['Coupon, %'].unique()[0]
    eurobonds_info = eurobonds_info.append(dict_bond, ignore_index=True)
coupons = coupons.dropna(axis=1, how='all')
coupons['Redemption of principal'] = coupons['Redemption of principal'].fillna(0)
coupons['payment'] = coupons['Coupon size'] + coupons['Redemption of principal']
coupons = coupons.rename(columns={'Coupon date':'date', 'Coupon, %':'rate'})

payments = coupons.pivot_table(index = 'Actual payment date', columns='isin', values='payment', aggfunc=np.sum)
payments = payments.fillna(0)
payments = payments[payments.index.sort_values(ascending=True)]
payments
eurobonds_info

### 2. Вычисление параметров моделей Нельсона-Сигеля и Нельсона-Сигеля-Свенссона по итогам каждого торгового дня в выбранном периоде. 


In [ ]:
#Create Nelson-Siegel spot rate function to optimize
def ns(m, omega):
    tau = omega[0]
    beta0 = omega[1]
    beta1 = omega[2]
    beta2 = omega[3]
    return beta0 + (beta1 + beta2) * (1 - np.exp(- m / tau)) / (m / tau) - beta2 * np.exp(- m / tau)
#Create Nelson-Siegel-Svensson spot rate function to optimize
def nss(m, omega):
    tau1 = omega[0]
    tau2 = omega[1]
    beta0 = omega[2]
    beta1 = omega[3]
    beta2 = omega[4]
    beta3 = omega[5]
    return beta0 + beta1 * (1 - np.exp(- m / tau1)) / (m / tau1) + beta2 * ((1 - np.exp(- m / tau1)) / (m / tau1) - np.exp(- m / tau1)) + beta3 * ((1 - np.exp(- m / tau2)) / (m / tau2) - np.exp(- m / tau2))

# Error functions initialization
def ns_sum_of_errors(omega, date):
    global eurobonds_info, eurobonds_prices, eurobonds_nkds, eurobonds_ytms, coupons, isins_list, bond_valuation, errors
    total_error = 0
    absolute_error_sum = 0
    sum_squared_error = 0
    bond_valuation = {}
    date_str = datetime.strftime(date, '%Y-%m-%d')
    today_isins = list(eurobonds_durations.loc[date_str][eurobonds_durations.loc[date_str] > 1].index)
    zcb_isins = list(eurobonds_durations.loc[date_str][eurobonds_durations.loc[date_str] < .5].index)

    for isin in today_isins:
        #current value of bonds' face value
        maturity = eurobonds_info[eurobonds_info['isin'] == isin]['maturity_date'].iloc[0]
        T = (maturity - date).days/365
        fv_current = 100 * np.exp(-ns(T, omega)*T/100)
        coupons_per_year = eurobonds_info[eurobonds_info['isin'] == isin]['coupon_payments_per_year'].iloc[0]
       
        #current value of coupons 
        bond_coupons = coupons[coupons['isin'] == isin]
        bond_coupons = bond_coupons[bond_coupons['date'] >= date]
        bond_coupons['years_to_coupon'] = bond_coupons['date'].apply(lambda x: (x-date).days/365)
        bond_coupons['zcb_rate'] = bond_coupons['years_to_coupon'].apply(lambda x: ns(x , omega))
        bond_coupons['discount_factor'] = bond_coupons['years_to_coupon'].apply(lambda x: np.exp(-ns(x, omega)*x/100))
        bond_coupons['current_value'] = bond_coupons['rate'] * bond_coupons['discount_factor']/coupons_per_year
        coupons_current = bond_coupons['current_value'].sum()
        bond_valuation[isin] = fv_current + coupons_current
        date_str = datetime.strftime(date, '%Y-%m-%d')
        bond_error = eurobonds_prices[isin][date_str] + eurobonds_nkds[isin][date_str]/eurobonds_info[eurobonds_info['isin'] == isin]['face_value'].iloc[0] - bond_valuation[isin]
        total_error += bond_error ** 2
#         total_error /= 1000
        absolute_error_sum += abs(bond_error)
        sum_squared_error += bond_error ** 2
#         print('PV(FV)=', fv_current)
#         print('PV(coupons)', coupons_current)
#         print('res=', bond_error)
#         print(bond_coupons)


        #omega penalty
    for isin in zcb_isins:
        bond_coupons = coupons[coupons['isin'] == isin]
        bond_coupons = bond_coupons[bond_coupons['date'] >= date]
        if len(bond_coupons) == 1:
            bond_coupons['years_to_coupon'] = bond_coupons['date'].apply(lambda x: (x-date).days/365)
            if abs(ns(bond_coupons['years_to_coupon'].iloc[0], omega) - eurobonds_ytms[isin][date_str]) > 0.05:
                total_error += 100000
                
    if omega[0] > 20 or omega[0] < 0:
        total_error += 100000
    if omega[1] + omega[2] < 0:
        total_error += 100000
    if omega[1] < 0 or omega[1] > 100:
        total_error += 100000
    if omega[2] < -100 or omega[2] > 100:
        total_error += 100000
    if omega[3] < -100 or omega[3] > 100:
        total_error += 100000
    
    errors['NS']['mae'][date] = absolute_error_sum / len(today_isins)
    errors['NS']['rmse'][date] =  (sum_squared_error / len(today_isins)) ** 0.5
    return total_error
def nss_sum_of_errors(omega, date):
    global eurobonds_info, eurobonds_prices, eurobonds_nkds, eurobonds_ytms, coupons, isins_list, bond_valuation, errors
    total_error = 0
    absolute_error_sum = 0
    sum_squared_error = 0
    bond_valuation = {}
    date_str = datetime.strftime(date, '%Y-%m-%d')
    today_isins = list(eurobonds_durations.loc[date_str][eurobonds_durations.loc[date_str] > 1].index)
    zcb_isins = list(eurobonds_durations.loc[date_str][eurobonds_durations.loc[date_str] < .5].index)
    for isin in today_isins:
        #current value of bonds' face value
        maturity = eurobonds_info[eurobonds_info['isin'] == isin]['maturity_date'].iloc[0]
        T = (maturity - date).days/365
        fv_current = 100 * np.exp(-nss(T, omega)*T/100)
        coupons_per_year = eurobonds_info[eurobonds_info['isin'] == isin]['coupon_payments_per_year'].iloc[0]
       
        #current value of coupons
        bond_coupons = coupons[coupons['isin'] == isin]
        bond_coupons['years_to_coupon'] = bond_coupons['date'].apply(lambda x: (x-date).days/365)
        bond_coupons['zcb_rate'] = bond_coupons['years_to_coupon'].apply(lambda x: nss(x , omega))
        bond_coupons['discount_factor'] = bond_coupons['years_to_coupon'].apply(lambda x: np.exp(-nss(x, omega)*x/100))
        bond_coupons['current_value'] = bond_coupons['rate'] * bond_coupons['discount_factor']/coupons_per_year
        coupons_current = bond_coupons['current_value'].sum()
        bond_valuation[isin] = fv_current + coupons_current
        bond_error = eurobonds_prices[isin][date_str] + eurobonds_nkds[isin][date_str]/eurobonds_info[eurobonds_info['isin'] == isin]['face_value'].iloc[0] - bond_valuation[isin]
        total_error += bond_error ** 2
#         total_error /= 1000
        absolute_error_sum += abs(bond_error)
        sum_squared_error += bond_error ** 2
#        print('PV(FV)=', fv_current)
#        print('PV(coupons)', coupons_current)
#        print('res=', bond_error)
#        print(bond_coupons)
        
        #omega penalty
    for isin in zcb_isins:
        bond_coupons = coupons[coupons['isin'] == isin]
        bond_coupons = bond_coupons[bond_coupons['date'] >= date]
        if len(bond_coupons) == 1:
            bond_coupons['years_to_coupon'] = bond_coupons['date'].apply(lambda x: (x-date).days/365)
            if abs(ns(bond_coupons['years_to_coupon'].iloc[0], omega) - eurobonds_ytms[isin][date_str]) > 0.05:
                total_error += 100000
    if omega[0] > 20 or omega[0] < 0:
        total_error += 100000
    if omega[1] > 20 or omega[1] < 0:
        total_error += 100000
    if omega[2] + omega[3] < 0:
        total_error += 100000
    if omega[2] < 0 or omega[2] > 100:
        total_error += 100000
    if omega[3] < -100 or omega[3] > 100:
        total_error += 100000
    if omega[4] < -100 or omega[4] > 100:
        total_error += 100000
    if omega[5] < -100 or omega[5] > 100:
        total_error += 100000
    
    errors['NSS']['mae'][date] = absolute_error_sum / len(today_isins)
    errors['NSS']['rmse'][date] =  (sum_squared_error / len(today_isins)) ** 0.5
    return total_error

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import minimize
results = {
    'NS':{'obj_fun':{}, 'x':{}, 'success':{}, 'nit':{}, 'time':{}},
    'NSS':{'obj_fun':{}, 'x':{}, 'success':{}, 'nit':{}, 'time':{}},
}
errors = {
    'NS':{'mae':{}, 'rmse':{}},
    'NSS':{'mae':{}, 'rmse':{}},
}

In [ ]:
#Historical data check

omega = {
    'NS':[10, 5.5, -3.5, 2],
    'NS_dw':[10, 5.5, -3.5, 2],
    'NSS':[10, 2, 5.5, -3.5, 2, 0],
    'NSS_dw':[10, 2, 5.5, -3.5, 2, 0]
}

absolute_errors = {}
day_number = 0
dates = [datetime.strptime('2018-03-29', '%Y-%m-%d')]
for date in list(eurobonds_prices.index):
# for date in dates:
#     date = datetime.strtime(date, '%Y-%m-%d')
    day_number += 1
    print(date, 'day number {} out of {}'.format(day_number, len(list(eurobonds_prices.index))))
    start = time.time()
    NS = minimize(ns_sum_of_errors, omega['NS'], args=(date), method='Nelder-Mead')
    results['NS']['obj_fun'][date] = NS.fun
    results['NS']['x'][date] = NS.x
    results['NS']['success'][date] = NS.success
    results['NS']['nit'][date] = NS.nit
    results['NS']['time'][date] = time.time() - start
    print('NS Equally_weighted:', NS.fun)
    print('Time:', time.time() - start)

    start = time.time()
    NSS = minimize(nss_sum_of_errors, omega['NSS'], args=(date), method='Nelder-Mead')
    results['NSS']['obj_fun'][date] = NSS.fun
    results['NSS']['x'][date] = NSS.x
    results['NSS']['success'][date] = NSS.success
    results['NSS']['nit'][date] = NSS.nit
    results['NSS']['time'][date] = time.time() - start
    print('NSS Equally_weighted:', NSS.fun)
    print('Time:', time.time() - start)

    omega['NS'] = NS.x
    omega['NSS'] = NSS.x

In [ ]:
# test
omega = {
    'NS':[10, 5.5, -3.5, 2],
    'NS_dw':[10, 5.5, -3.5, 2],
    'NSS':[10, 2, 5.5, -3.5, 2, 0],
    'NSS_dw':[10, 2, 5.5, -3.5, 2, 0]
}
date = datetime.strptime('2018-01-01', '%Y-%m-%d')
# ns_sum_of_errors(omega['NS'], date)
NS = minimize(ns_sum_of_errors, omega['NS'], args=(date), method='Nelder-Mead')
NS

### 3. Сохранение данных в форматах .json и .txt.

In [ ]:
#Saving results

import json
trydict = results['NS_dw']['obj_fun'].copy()
def keys_to_str(dictionary):
    dict = dictionary.copy()
    for key in list(dict.keys()):
        if type(key) is not str:
            try:
                dict[datetime.strftime(key, '%Y-%m-%d')] = dict[key]
            except:
                try:
                    dict[repr(key)] = dict[key]
                except:
                    pass
            del dict[key]
    return dict

np.set_printoptions(suppress=True)
results_dump = {
    'NS':{'obj_fun':{}, 'x':{}, 'success':{}, 'nit':{}, 'time':{}},
    'NS_dw':{'obj_fun':{}, 'x':{}, 'success':{}, 'nit':{}, 'time':{}},
    'NSS':{'obj_fun':{}, 'x':{}, 'success':{}, 'nit':{}, 'time':{}},
    'NSS_dw':{'obj_fun':{}, 'x':{}, 'success':{}, 'nit':{}, 'time':{}}
}
results_dump['NS']['obj_fun'] = keys_to_str(results['NS']['obj_fun'])
results_dump['NS']['x'] = keys_to_str(results['NS']['x'])
results_dump['NS']['success'] = keys_to_str(results['NS']['success'])
results_dump['NS']['nit'] = keys_to_str(results['NS']['nit'])
results_dump['NS']['time'] = keys_to_str(results['NS']['time'])
results_dump['NSS']['obj_fun'] = keys_to_str(results['NS']['obj_fun'])
results_dump['NSS']['x'] = keys_to_str(results['NSS']['x'])
results_dump['NSS']['success'] = keys_to_str(results['NSS']['success'])
results_dump['NSS']['nit'] = keys_to_str(results['NSS']['nit'])
results_dump['NSS']['time'] = keys_to_str(results['NSS']['time'])
results_dump['NS_dw']['obj_fun'] = keys_to_str(results['NS_dw']['obj_fun'])
results_dump['NS_dw']['x'] = keys_to_str(results['NS_dw']['x'])
results_dump['NS_dw']['success'] = keys_to_str(results['NS_dw']['success'])
results_dump['NS_dw']['nit'] = keys_to_str(results['NS_dw']['nit'])
results_dump['NS_dw']['time'] = keys_to_str(results['NS_dw']['time'])
results_dump['NSS_dw']['obj_fun'] = keys_to_str(results['NS_dw']['obj_fun'])
results_dump['NSS_dw']['x'] = keys_to_str(results['NSS_dw']['x'])
results_dump['NSS_dw']['success'] = keys_to_str(results['NSS_dw']['success'])
results_dump['NSS_dw']['nit'] = keys_to_str(results['NSS_dw']['nit'])
results_dump['NSS_dw']['time'] = keys_to_str(results['NSS_dw']['time'])
errors_dump = {
    'NS':{'mae':{}, 'rmse':{}},
    'NS_dw':{'mae':{}, 'rmse':{}},
    'NSS':{'mae':{}, 'rmse':{}},
    'NSS_dw':{'mae':{}, 'rmse':{}}
}
errors_dump['NS']['mae'] = keys_to_str(errors['NS']['mae'])
errors_dump['NS']['rmse'] = keys_to_str(errors['NS']['rmse'])
errors_dump['NSS']['mae'] = keys_to_str(errors['NSS']['mae'])
errors_dump['NSS']['rmse'] = keys_to_str(errors['NSS']['rmse'])
errors_dump['NS_dw']['mae'] = keys_to_str(errors['NS_dw']['mae'])
errors_dump['NS_dw']['rmse'] = keys_to_str(errors['NS_dw']['rmse'])
errors_dump['NSS_dw']['mae'] = keys_to_str(errors['NSS_dw']['mae'])
errors_dump['NSS_dw']['rmse'] = keys_to_str(errors['NSS_dw']['rmse'])
# with open('errorsNSS.json', 'w') as outfile:
#     json.dump(errors_dump, outfile, indent=4)
# with open('results_dumpNM_rus.json', 'w') as outfile:
#     json.dump(results_dump, outfile, indent=4, default=str)
with open('errorsNSS.txt', 'w') as outfile:
    json.dump(errors_dump, outfile,indent=4)
# with open('results_dumpNM_rus.txt', 'w') as outfile:
#     json.dump(results_dump, outfile, indent=4, default=str)

### 4. Графическое представление результатов – структура кривых, динамика отдельных ставок, динамика ошибок. 

In [ ]:
# Put everything on graphs

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from matplotlib import cm
t = np.linspace(0, 20, 1000)
curves_NS = np.array([ns(t, results['NS']['x'][date]) for date in sorted(results['NS']['x'].keys())])
z = np.array([x for x in range(1, 102)])
t, z = np.meshgrid(t, z)
print(np.shape(t), np.shape(z), np.shape(curves_NS))
fig = plt.figure(figsize=(20, 15))
ax = fig.gca(projection='3d')
gr = ax.plot_surface(z, t, curves_NS, cmap=cm.viridis, vmin=1, vmax=6)
ax.set_zbound([.5, 9])
fig.colorbar(gr, shrink=0.25, aspect=5)
ax.set_xmargin(0)
ax.set_ymargin(0)
ax.set_zmargin(0)
ax.set(title='NS curves structure', xlabel='Date', ylabel="Maturity", zlabel='Spot rate')
ax.set_xticklabels([datetime.strftime(x, '%Y-%m-%d') for x in list(eurobonds_prices.index)][42:142:4])
ax.view_init(15, -45)


In [ ]:
len(curves_NS)

In [ ]:
t = np.linspace(0, 20, 1000)
curves_NSS = np.array([nss(t, results['NSS']['x'][date]) for date in sorted(results['NSS']['x'].keys())])
z = np.array([x for x in range(1, 102)])
t, z = np.meshgrid(t, z)
fig = plt.figure(figsize=(20, 15))
ax = fig.gca(projection='3d')
gr = ax.plot_surface(z, t, curves_NSS, cmap=cm.magma, vmin=1, vmax=6)
ax.set_zbound([.5, 7.5])
fig.colorbar(gr, shrink=0.3, aspect=5)
ax.set_xmargin(0)
ax.set_ymargin(0)
ax.set_zmargin(0)
ax.set(title='NSS curves structure', xlabel='Date', ylabel="Maturity", zlabel='Spot rate')
ax.set_xticklabels([datetime.strftime(x, '%Y-%m-%d') for x in list(eurobonds_prices.index)][0::85])
ax.view_init(45, -45)

In [ ]:
m0points_NS = {}
m0points_NSS = {}
m5points_NS = {}
m5points_NSS = {}
m10points_NSS = {}
m10points_NS = {}

In [ ]:
for date in eurobonds_prices.index[42:142]:
    m0points_NS[datetime.strftime(date, '%Y-%m-%d')] = results['NS']['x'][date][1] + results['NS']['x'][date][2]
    m5points_NS[datetime.strftime(date, '%Y-%m-%d')] = ns(5, results['NS']['x'][date])
    m10points_NS[datetime.strftime(date, '%Y-%m-%d')] = ns(10, results['NS']['x'][date])
#     m0points_NS_dw[datetime.strftime(date, '%Y-%m-%d')] = results['NS_dw']['x'][date][1] + results['NS_dw']['x'][date][2]
#     m5points_NS_dw[datetime.strftime(date, '%Y-%m-%d')] = ns(5, results['NS_dw']['x'][date])
#     m10points_NS_dw[datetime.strftime(date, '%Y-%m-%d')] = ns(10, results['NS_dw']['x'][date])
    m0points_NSS[datetime.strftime(date, '%Y-%m-%d')] = results['NSS']['x'][date][2] + results['NSS']['x'][date][3]
    m5points_NSS[datetime.strftime(date, '%Y-%m-%d')] = nss(5, results['NSS']['x'][date])
    m10points_NSS[datetime.strftime(date, '%Y-%m-%d')] = nss(10, results['NSS']['x'][date])

fig = plt.figure(num=None, figsize=(10,10), dpi=80, facecolor='w', edgecolor='k')

axes1 = fig.add_axes([0, 0, 1, 1])
axes1.plot(*zip(*sorted(m0points_NS.items())), label='NS')
# axes1.plot(*zip(*sorted(m0points_NS_dw.items())), label='NS_dw')
axes1.plot(*zip(*sorted(m0points_NSS.items())), label='NSS')

axes1.legend(fontsize=15)
axes1.grid()
axes1.set_xticks(ticks=range(0,len(m0points_NS.keys()), 70))
fig.suptitle('Overnight rates r(0) from Nelder-Mead optimization', fontsize=16, y=1.03)
axes1.set_ylim([0, 10])


In [ ]:
fig = plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
axes1 = fig.add_axes([0, 0, 1, 1])
axes1.plot(*zip(*sorted(m10points_NS.items())), label='NS 10-year')
axes1.plot(*zip(*sorted(m10points_NSS.items())), label='NSS 10-year')
axes1.plot(*zip(*sorted(m5points_NS.items())), label='NS 5-year')
axes1.plot(*zip(*sorted(m5points_NSS.items())), label='NSS 5-year')
# axes1.plot(*zip(*sorted(m0points_NS.items())), label='NS overnight')
# axes1.plot(*zip(*sorted(m0points_NSS.items())), label='NSS overnight')

axes1.legend(fontsize=15)
axes1.grid()
axes1.set_xticks(ticks=range(0,len(m0points_NS.keys()), 70))
fig.suptitle('Some zero-coupon rates from Nelder-Mead optimization', fontsize=16, y=1.03)

In [ ]:
eurobonds_prices.isna().sum()